In [1]:
import os
import urllib
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_addons as tfa
import time

from random import seed
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(tf.test.gpu_device_name())

2.6.1
/device:GPU:0


# Read the data

In [2]:
def load_UCI(dset_name, splits=10, seed=0, separate_targets=True, save_dir='data/'):
    mkdir(save_dir)

    if dset_name == 'boston':
        if not os.path.isfile(save_dir+'housing.data'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
                               filename=save_dir+'housing.data')
        data = pd.read_csv(save_dir + 'housing.data', header=0, delimiter="\s+").values
        y_idx = [-1]

    elif dset_name == 'concrete':
        if not os.path.isfile(save_dir+'Concrete_Data.xls'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls",
                               filename=save_dir+'Concrete_Data.xls')
        data = pd.read_excel(save_dir+ 'Concrete_Data.xls', header=0, delimiter="\s+").values
        y_idx = [-1]

    elif dset_name == 'energy':
        if not os.path.isfile(save_dir+'ENB2012_data.xlsx'):
            urllib.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx",
                               filename=save_dir+'ENB2012_data.xlsx')
        data = pd.read_excel(save_dir+'ENB2012_data.xlsx', header=0, delimiter="\s+").values
        y_idx = [-2, -1]

    elif dset_name == 'power':
        if not os.path.isfile(save_dir+'CCPP.zip'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip",
                               filename=save_dir+'CCPP.zip')
        zipped = zipfile.ZipFile(save_dir+"CCPP.zip")
        data = pd.read_excel(zipped.open('CCPP/Folds5x2_pp.xlsx'), header=0, delimiter="\t").values
        y_idx = [-1]

    elif dset_name == 'wine':
        if not os.path.isfile(save_dir+'winequality-red.csv'):
            urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
                               filename=save_dir+'winequality-red.csv')
        data = pd.read_csv(save_dir+'winequality-red.csv', header=1, delimiter=';').values
        y_idx = [-1]

    elif dset_name == 'yatch':
        if not os.path.isfile(save_dir+'yacht_hydrodynamics.data'):
            urllib.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",
                               filename=save_dir+'yacht_hydrodynamics.data')
        data = pd.read_csv(save_dir+'yacht_hydrodynamics.data', header=1, delimiter='\s+').values
        y_idx = [-1]

    elif dset_name == 'kin8nm':
        if not os.path.isfile(save_dir+'dataset_2175_kin8nm.csv'):
            urllib.urlretrieve("https://www.openml.org/data/get_csv/3626/dataset_2175_kin8nm.csv",
                               filename=save_dir+'dataset_2175_kin8nm.csv')
        data = pd.read_csv(save_dir+'dataset_2175_kin8nm.csv', header=1, delimiter=',').values
        y_idx = [-1]

    elif dset_name == 'kin8nm':
        if not os.path.isfile(save_dir+'dataset_2175_kin8nm.csv'):
            urllib.urlretrieve("https://www.openml.org/data/get_csv/3626/dataset_2175_kin8nm.csv",
                               filename=save_dir+'dataset_2175_kin8nm.csv')
        data = pd.read_csv(save_dir+'dataset_2175_kin8nm.csv', header=1, delimiter=',').values
        y_idx = [-1]

    elif dset_name == 'naval':
        if not os.path.isfile(save_dir + 'UCI%20CBM%20Dataset.zip'):
            urllib.urlretrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/00316/UCI%20CBM%20Dataset.zip",
                               filename=save_dir + 'UCI%20CBM%20Dataset.zip')
        zipped = zipfile.ZipFile(save_dir + "UCI%20CBM%20Dataset.zip")
        data = pd.read_csv(zipped.open('UCI CBM Dataset/data.txt'), header='infer', delimiter="\s+").values
        y_idx = [-2, -1]

    elif dset_name == 'protein':
        if not os.path.isfile(save_dir+'CASP.csv'):
            urllib.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv",
                               filename=save_dir+'CASP.csv')
        data = pd.read_csv(save_dir+'CASP.csv', header=1, delimiter=',').values
        y_idx = [0]

    elif dset_name == 'default_credit':
        if not os.path.isfile(save_dir + 'default of credit card clients.xls'):
            urllib.request.urlretrieve(
                "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
                filename=save_dir + 'default of credit card clients.xls')
        data = pd.read_excel(save_dir + 'default of credit card clients.xls', header=[0, 1], index_col=0, # delimiter="\s+"
                             ).values
        y_idx = [-1]  # OK


    else:
        raise Exception('Dataset name doesnt match any known datasets.')

    np.random.seed(seed)
    data = data[np.random.permutation(np.arange(len(data)))]

    kf = KFold(n_splits=splits)
    for j, (train_index, test_index) in enumerate(kf.split(data)):

        if separate_targets:
            x_idx = list(range(data.shape[1]))
            for e in y_idx:
                x_idx.remove(x_idx[e])

            x_idx = np.array(x_idx)
            y_idx = np.array(y_idx)
            x_train, y_train = data[train_index, :], data[train_index, :]
            x_train, y_train = x_train[:, x_idx], y_train[:, y_idx]
            x_test, y_test = data[test_index, :], data[test_index, :]
            x_test, y_test = x_test[:, x_idx], y_test[:, y_idx]

            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)
            y_means, y_stds = y_train.mean(axis=0), y_train.std(axis=0)

            y_stds[y_stds < 1e-10] = 1
            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            y_train = ((y_train - y_means) / y_stds).astype(np.float32)

            x_test = ((x_test - x_means) / x_stds).astype(np.float32)
            y_test = ((y_test - y_means) / y_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds

        else:
            x_train, x_test = data[train_index, :], data[test_index, :]
            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)

            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            x_test = ((x_test - x_means) / x_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds

def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path, mode=0o777)
            
def unnormalise_cat_vars(x, x_means, x_stds, input_dim_vec):
    input_dim_vec = np.array(input_dim_vec)
    unnorm_x = np.multiply(x, x_stds) + x_means

    fixed_unnorm = unnorm_x.round()
    fixed_unnorm -= fixed_unnorm.min(axis=0).reshape([1, fixed_unnorm.shape[1]])  # this sets all mins to 0
    for idx, dims in enumerate(input_dim_vec):
        if dims > 1:
            vec = fixed_unnorm[:, idx]
            vec[vec > dims - 1] = dims - 1
            fixed_unnorm[:, idx] = vec

    x[:, input_dim_vec > 1] = fixed_unnorm[:, input_dim_vec > 1]
    return x

In [3]:
def get_my_COMPAS(rseed=0, separate_test=True, test_ratio=0.2, save_dir='../data/'):
    """
        The adult dataset can be obtained from: https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
        The code will look for the data file in the present directory, if it is not found, it will download them from GitHub.
    """

    SEED = rseed
    seed(SEED)
    np.random.seed(SEED)

    their_FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"]
    FEATURES_CLASSIFICATION = ["age_cat", "race", "sex", "c_charge_degree", "is_recid", "priors_count",
                               "time_served"]  # features to be used for classification
    CONT_VARIABLES = ["priors_count",
                      "time_served"]  # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "two_year_recid"  # the decision variable


    COMPAS_INPUT_FILE = save_dir + "compas-scores-two-years.csv"
    check_data_file(COMPAS_INPUT_FILE)

    # load the data and get some stats
    df = pd.read_csv(COMPAS_INPUT_FILE)
    df = df.dropna(subset=["days_b_screening_arrest"])  # dropping missing vals

    # convert to np array
    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])

    dates_in = data['c_jail_in']
    dates_out = data['c_jail_out']
    # this measures time in Jail
    time_served = []
    for i in range(len(dates_in)):
        di = datetime.datetime.strptime(dates_in[i], '%Y-%m-%d %H:%M:%S')
        do = datetime.datetime.strptime(dates_out[i], '%Y-%m-%d %H:%M:%S')
        time_served.append((do - di).days)
    time_served = np.array(time_served)
    time_served[time_served < 0] = 0
    data["time_served"] = time_served

    """ Filtering the data """

    # These filters are the same as propublica (refer to https://github.com/propublica/compas-analysis)
    # If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, we assume that because of data quality reasons, that we do not have the right offense.
    idx = np.logical_and(data["days_b_screening_arrest"] <= 30, data["days_b_screening_arrest"] >= -30)

    # We coded the recidivist flag -- is_recid -- to be -1 if we could not find a compas case at all.
    idx = np.logical_and(idx, data["is_recid"] != -1)

    # In a similar vein, ordinary traffic offenses -- those with a c_charge_degree of 'O' -- will not result in Jail time are removed (only two of them).
    idx = np.logical_and(idx, data["c_charge_degree"] != "O")  # F: felony, M: misconduct

    # We filtered the underlying data from Broward county to include only those rows representing people who had either recidivated in two years, or had at least two years outside of a correctional facility.
    idx = np.logical_and(idx, data["score_text"] != "NA")

    # select the examples that satisfy this criteria
    for k in data.keys():
        data[k] = data[k][idx]

    y = data[CLASS_FEATURE]
    """ Feature normalization and one hot encoding """

    print
    "\nNumber of people recidivating within two years"
    print
    pd.Series(y).value_counts()
    print
    "\n"

    X = []  # empty array with num rows same as num examples, will hstack the features to it
    X_dims = []

    feature_names = []
    for attr in FEATURES_CLASSIFICATION:
        vals = data[attr]
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            # vals = preprocessing.scale(vals, axis=0, with_mean=True, with_std=True)  # 0 mean and 1 variance
            vals = np.reshape(vals, (len(y), -1))  # convert from 1-d arr to a 2-d arr with one col
            X_dims.append(1)

        else:  # for binary categorical variables, the label binarizer uses just one var instead of two
            enc = preprocessing.OneHotEncoder(categories='auto', handle_unknown='error')
            enc.fit(vals.reshape(-1, 1))
            vals = enc.transform(vals.reshape(-1, 1)).todense()
            X_dims += [vals.shape[1]]*vals.shape[1]

        # add to learnable features
        X.append(vals)

        if attr in CONT_VARIABLES:  # continuous feature, just append the name
            feature_names.append(attr)
        else:  # categorical features
            if vals.shape[1] == 1:  # binary features that passed through lib binarizer
                feature_names.append(attr)
            else:
                for k in enc.categories_:  # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))

    X = np.array(np.concatenate(list(X), axis=1))
    X_dims = np.array(X_dims)

    if separate_test:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=rseed, shuffle=True)

        x_means, x_stds = X_train.mean(axis=0), X_train.std(axis=0)
        x_means[X_dims>1] = 0
        x_stds[X_dims>1] = 1
        x_stds[x_stds < 1e-10] = 1

        x_train = ((X_train - x_means) / x_stds).astype(np.float32)
        x_test = ((X_test - x_means) / x_stds).astype(np.float32)

        return x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims
    else:
        x_means, x_stds = X.mean(axis=0), X.std(axis=0)
        print(X_dims.shape, x_means.shape)
        x_means[:,X_dims>1] = 0
        x_stds[:,X_dims>1] = 1
        x_stds[x_stds < 1e-10] = 1

        x_train = ((X - x_means) / x_stds).astype(np.float32)

        return x_train, x_means, x_stds, y, feature_names, X_dims

def check_data_file(fname):
    files = os.listdir(".")  # get the current directory listing
    print
    "Looking for file '%s' in the current directory..." % fname

    if fname not in files:
        print
        "'%s' not found! Downloading from GitHub..." % fname
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        
        response = urllib.request.urlopen(addr)

        data = response.read()
        fileOut = open(fname, "wb")
        fileOut.write(data)
        fileOut.close()
        print
        "'%s' download and saved locally.." % fname
    else:
        print
        "File found in current directory.."


def X_dims_to_input_dim_vec(X_dims):
    """This is for our cat_Gauss VAE model"""
    input_dim_vec = []
    i = 0
    while i < len(X_dims):
        input_dim_vec.append(X_dims[i])
        i += X_dims[i]
    return np.array(input_dim_vec)

In [4]:
names = ['wine', 'default_credit', 'compas', 'lsat']
widths = [300, 300, 300, 300] # [200, 200, 200, 200]
depths = [3, 3, 3, 3] # We go deeper because we are using residual models
latent_dims = [6, 8, 4, 4]

# x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds = \
#     load_UCI(dset_name='wine', splits=10, seed=42, separate_targets=True, save_dir='../data/')
# print('Wine', x_train.shape, x_test.shape)

x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds = \
load_UCI(dset_name='default_credit', splits=10, seed=42, separate_targets=True, save_dir='../data/')
input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1]

x_train = unnormalise_cat_vars(x_train, x_means, x_stds, input_dim_vec)
x_test = unnormalise_cat_vars(x_test, x_means, x_stds, input_dim_vec)
# target unnormalisation
y_train = unnormalise_cat_vars(y_train, y_means, y_stds, [2])
y_test = unnormalise_cat_vars(y_test, y_means, y_stds, [2])

dname = 'default_credit'
print(dname)

default_credit


# Define the model

## Encoder

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_encoder(input_dim_vec,width,depth,latent_dim):
    inputs = keras.Input(shape=(sum(input_dim_vec),))
    input = layers.Dense(width)(inputs)

    for i in range(depth-1):

        x = layers.LeakyReLU()(input)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(width, kernel_initializer=tf.keras.initializers.Zeros())(x)

        x = x + input

        input = x

    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(latent_dim*2,kernel_initializer=tf.keras.initializers.Zeros())(x)

    encoder = keras.Model(inputs=inputs, outputs=outputs, name="encoder_model")

    # encoder.summary()

    # keras.utils.plot_model(encoder, "encoder.png")
    keras.utils.plot_model(encoder, "encoder.png", show_shapes=True)
    
    return encoder

## Decoder

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def create_decoder(input_dim_vec,width,depth,latent_dim):

    inputs = keras.Input(shape=(latent_dim,))
    input = layers.Dense(width,kernel_initializer=tf.keras.initializers.Zeros())(inputs)
    input = layers.LeakyReLU()(input)
    input = layers.BatchNormalization()(input)

    for i in range(depth-1):

        x = layers.LeakyReLU()(input)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(width,kernel_initializer=tf.keras.initializers.Zeros())(x)

        x = x + input

        input = x

    outputs = layers.Dense(sum(input_dim_vec),kernel_initializer=tf.keras.initializers.Zeros())(x)

    decoder = keras.Model(inputs=inputs, outputs=outputs, name="decoder_model")

    # decoder.summary()

    # keras.utils.plot_model(model, "decoder_model.png")
    keras.utils.plot_model(decoder, "decoder.png", show_shapes=True)
    
    return decoder

## Summary of encoder
#### Sequential network:
#### input = linear (FC)
#### body = SkipConnection (depth-1) times
###### 1 SkipConnection = LeaklyRelu + BatchNorm1d + Linear
#### output = LeakyReLU + BatchNorm1d + Linear

# Class VAE and loss and Training

In [7]:
print(f"x_train={x_train.shape} x_test={x_test.shape}")

x_train=(27000, 23) x_test=(3000, 23)


In [8]:
class VAE(tf.keras.Model):
    def __init__(self, input_dim_vec,width,depth,latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = create_encoder(input_dim_vec,width,depth,latent_dim)
        self.decoder = create_decoder(input_dim_vec,width,depth,latent_dim)

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)
    
#     def normal_parse_params(params, min_sigma=1e-3):
#         n = params.shape[0]
#         d = params.shape[1]
#         mu = params[:, :d // 2]
#         sigma_params = params[:, d // 2:]
#         sigma = tf.math.softplus(sigma_params)
#         sigma = tf.clip_by_value(sigma, clip_value_min=1e-3, clip_value_max=1e10)
#         dist = tfp.distributions.Normal(loc=mu, scale=sigma)
#         return dist

#     def encode(x):
#         """Works with flattened representATION"""
#         approx_post_params = encoder(x)
#         approx_post = normal_parse_params(approx_post_params, 1e-3)
#         return approx_post

#     def decode(z_sample):
#         """Works with flattened representATION"""
#         rec_params = decoder(z_sample)
#         return rec_params
    
    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar
    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits
    
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),axis=raxis)

def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent)#, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

@tf.function
def train_step(model, x, optimizer):

    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [9]:
# optimizer = tf.keras.optimizers.Adam(1e-4)
optimizer = tfa.optimizers.RectifiedAdam(1e-4)

In [20]:
def train_VAE(model,train_dataset,test_dataset, epochs):
    for epoch in range(1, epochs + 1):
        start_time = time.time()

        ## Training
        # for i,x in enumerate(x_train):
        loss = tf.keras.metrics.Mean()
        for i,(x,y) in enumerate(train_dataset.as_numpy_iterator()):
            # x = x.reshape(len[x],len(x[0]))
            x_flat = gauss_cat_to_flat(x, input_dim_vec)
            train_step(model, x_flat, optimizer)
            print(f"Finished {i}/{len(x_train)}",end="\r")
        
        ## Testing
        loss = tf.keras.metrics.Mean()
        # for x in x_test:
        nb_samples = 0
        vlb = 0
        for x,y in test_dataset.as_numpy_iterator():
            # x = x.reshape(1,len(x)) # reshape(1,23)
            x_flat = gauss_cat_to_flat(x, input_dim_vec)
            
            cost = compute_loss(model, x_flat)
            vlb += cost# * len(x)
            nb_samples += len(x)
            # loss(compute_loss(model, x_flat))
        vlb /= -nb_samples
        #elbo = -loss.result()
        end_time = time.time()
        
        print(f'Epoch: {epoch} Test set elbo={vlb}, time elapse for current epoch: {end_time - start_time}')

In [11]:
def gauss_cat_to_flat(x, input_dim_vec):
    output = []
    for idx, dim in enumerate(input_dim_vec):
        if dim == 1:
            output.append(tf.expand_dims(x[:, idx], axis=1))
        elif dim > 1:            
            oh_vec = tf.one_hot(x[:, idx], dim)
            output.append(oh_vec)
        else:
            raise ValueError('Error, invalid dimension value')
    return tf.concat(output, axis=1).numpy()

# Default_credit

In [12]:
input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1]
width = 300
depth = 3
latent_dim = 8

In [13]:
epochs = 10
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 8
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
# random_vector_for_generation = tf.random.normal(
#     shape=[num_examples_to_generate, latent_dim])
model = VAE(input_dim_vec,width,depth,latent_dim)

In [14]:
train_VAE(model)

1


KeyboardInterrupt: 

# Compas

In [12]:
x_train, x_test, x_means, x_stds, y_train, y_test, feature_names, X_dims = \
    get_my_COMPAS(rseed=42, separate_test=True, test_ratio=0.1, save_dir='../data/')
input_dim_vec = X_dims_to_input_dim_vec(X_dims)
print('Compas', x_train.shape, x_test.shape)
print(input_dim_vec)


dname = 'compas'
print(dname)

Compas (5554, 17) (618, 17)
[3 6 2 2 2 1 1]
compas


In [13]:
width = widths[names.index(dname)]
depth = depths[names.index(dname)] # number of hidden layers
latent_dim = latent_dims[names.index(dname)]

num_examples_to_generate = 16

batch_size = 128
nb_epochs = 2500
lr = 1e-4
early_stop = 200

# cuda = torch.cuda.is_available()

In [14]:
model = VAE(input_dim_vec,width,depth,latent_dim)

In [15]:
trainX = tf.convert_to_tensor(x_train)
trainy = tf.convert_to_tensor(y_train)

testX = tf.convert_to_tensor(x_test)
testy = tf.convert_to_tensor(y_test)

train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainy)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(batch_size)

In [ ]:
# net.load(save_dir+'_models/theta_best.dat')

# vlb_train, vlb_dev = train_VAE(net, save_dir, batch_size, nb_epochs, trainset, valset,
#                                cuda=cuda, flat_ims=False, train_plot=False, early_stop=early_stop)

train_VAE(model,train_dataset,test_dataset, nb_epochs)

Epoch: 1 Test set elbo=-11.30302906036377, time elapse for current epoch: 0.44735050201416016
Epoch: 2 Test set elbo=-11.287198066711426, time elapse for current epoch: 0.4417424201965332
Epoch: 3 Test set elbo=-11.271244049072266, time elapse for current epoch: 0.4086785316467285
Epoch: 4 Test set elbo=-11.255175590515137, time elapse for current epoch: 0.42075300216674805
Epoch: 5 Test set elbo=-11.238995552062988, time elapse for current epoch: 0.39911341667175293
Epoch: 6 Test set elbo=-11.222712516784668, time elapse for current epoch: 0.38346028327941895
Epoch: 7 Test set elbo=-11.206332206726074, time elapse for current epoch: 0.4319443702697754
Epoch: 8 Test set elbo=-11.189847946166992, time elapse for current epoch: 0.39142894744873047
Epoch: 9 Test set elbo=-11.173293113708496, time elapse for current epoch: 0.41916871070861816
Epoch: 10 Test set elbo=-11.156654357910156, time elapse for current epoch: 0.4442868232727051
Epoch: 11 Test set elbo=-11.13992691040039, time elaps

Epoch: 88 Test set elbo=-9.841670036315918, time elapse for current epoch: 0.5095641613006592
Epoch: 89 Test set elbo=-9.826066017150879, time elapse for current epoch: 0.5224704742431641
Epoch: 90 Test set elbo=-9.810516357421875, time elapse for current epoch: 0.4867074489593506
Epoch: 91 Test set elbo=-9.795005798339844, time elapse for current epoch: 0.5234835147857666
Epoch: 92 Test set elbo=-9.779529571533203, time elapse for current epoch: 0.459653377532959
Epoch: 93 Test set elbo=-9.764122009277344, time elapse for current epoch: 0.45585131645202637
Epoch: 94 Test set elbo=-9.74875545501709, time elapse for current epoch: 0.45282959938049316
Epoch: 95 Test set elbo=-9.733433723449707, time elapse for current epoch: 0.43091726303100586
Epoch: 96 Test set elbo=-9.718165397644043, time elapse for current epoch: 0.4533078670501709
Epoch: 97 Test set elbo=-9.702940940856934, time elapse for current epoch: 0.4552011489868164
Epoch: 98 Test set elbo=-9.687758445739746, time elapse for

Epoch: 175 Test set elbo=-8.663049697875977, time elapse for current epoch: 0.557112455368042
Epoch: 176 Test set elbo=-8.651546478271484, time elapse for current epoch: 0.5063052177429199
Epoch: 177 Test set elbo=-8.64007568359375, time elapse for current epoch: 0.5651886463165283
Epoch: 178 Test set elbo=-8.628658294677734, time elapse for current epoch: 0.5044660568237305
Epoch: 179 Test set elbo=-8.617276191711426, time elapse for current epoch: 0.4800431728363037
Epoch: 180 Test set elbo=-8.605948448181152, time elapse for current epoch: 0.43071770668029785
Epoch: 181 Test set elbo=-8.59465503692627, time elapse for current epoch: 0.4478597640991211
Epoch: 182 Test set elbo=-8.583403587341309, time elapse for current epoch: 0.4645366668701172
Epoch: 183 Test set elbo=-8.572195053100586, time elapse for current epoch: 0.45181798934936523
Epoch: 184 Test set elbo=-8.561041831970215, time elapse for current epoch: 0.4771111011505127
Epoch: 185 Test set elbo=-8.549911499023438, time e

Epoch: 262 Test set elbo=-7.813146114349365, time elapse for current epoch: 0.47835683822631836
Epoch: 263 Test set elbo=-7.8050217628479, time elapse for current epoch: 0.47969818115234375
Epoch: 264 Test set elbo=-7.796916961669922, time elapse for current epoch: 0.4531669616699219
Epoch: 265 Test set elbo=-7.788848400115967, time elapse for current epoch: 0.4469034671783447
Epoch: 266 Test set elbo=-7.780825138092041, time elapse for current epoch: 0.4497048854827881
Epoch: 267 Test set elbo=-7.772826671600342, time elapse for current epoch: 0.5779962539672852
Epoch: 268 Test set elbo=-7.764859199523926, time elapse for current epoch: 0.5804264545440674
Epoch: 269 Test set elbo=-7.7569260597229, time elapse for current epoch: 0.6358950138092041
Epoch: 270 Test set elbo=-7.749039173126221, time elapse for current epoch: 0.6572558879852295
Epoch: 271 Test set elbo=-7.741166591644287, time elapse for current epoch: 0.6253423690795898
Epoch: 272 Test set elbo=-7.733343124389648, time el

Epoch: 349 Test set elbo=-7.218863010406494, time elapse for current epoch: 0.4876821041107178
Epoch: 350 Test set elbo=-7.213188648223877, time elapse for current epoch: 0.46871519088745117
Epoch: 351 Test set elbo=-7.207546234130859, time elapse for current epoch: 0.4744541645050049
Epoch: 352 Test set elbo=-7.201927185058594, time elapse for current epoch: 0.4640390872955322
Epoch: 353 Test set elbo=-7.196322917938232, time elapse for current epoch: 0.4767429828643799
Epoch: 354 Test set elbo=-7.190742492675781, time elapse for current epoch: 0.4706110954284668
Epoch: 355 Test set elbo=-7.1851911544799805, time elapse for current epoch: 0.4627687931060791
Epoch: 356 Test set elbo=-7.179653644561768, time elapse for current epoch: 0.48433780670166016
Epoch: 357 Test set elbo=-7.174144268035889, time elapse for current epoch: 0.48667359352111816
Epoch: 358 Test set elbo=-7.168660640716553, time elapse for current epoch: 0.4531095027923584
Epoch: 359 Test set elbo=-7.1631879806518555, 

In [ ]:
model.save_weights('./checkpoints/vae')